In [1]:
# ruff: noqa: F401, E402, T201, T203, D103

from __future__ import annotations

import math
import sys
import time
from bisect import bisect_left, bisect_right
from collections import Counter, deque
from functools import cmp_to_key, lru_cache, reduce
from heapq import heapify, heappop, heappush, heappushpop, nsmallest
from importlib import reload
from itertools import (
    accumulate,
    chain,
    combinations,
    islice,
    pairwise,
    permutations,
    product,
    repeat,
    starmap,
    tee,
)
from math import comb, exp, factorial, inf, log, prod, sqrt
from operator import add, and_, contains, indexOf, itemgetter, mul, neg, or_, xor
from pprint import pformat, pprint
from typing import Callable, Iterable, Iterator, List, Tuple

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from IPython.display import HTML, clear_output, display

import arrays
import bits
import combinatorics
import graphs
import grid
import lists
import mathematics
import matrix
import parsing
import search
import sequences
import sets
import stack
import strings
import trees

for m in (
    arrays,
    bits,
    combinatorics,
    graphs,
    grid,
    lists,
    mathematics,
    matrix,
    parsing,
    search,
    sequences,
    sets,
    stack,
    strings,
    trees,
):
    reload(m)


from combinatorics import fib
from mathematics import prime_numbers as primes
from search import lower_index, upper_index
from sequences import find_if
from strings import splint

np.set_printoptions(linewidth=110)
print()

In [16]:
a = np.tile(np.arange(2), 6)
b = np.tile(np.arange(3), 4)
c = np.tile(np.arange(4), 3)
d = np.tile(np.arange(6), 2)
print(f"{0}: \t {a} \t {b} \t {c} \t {d}")
for i in primes(20):
    print(f"{i}: \t {a % i} \t {b % i} \t {c % i} \t {d % i}")

0: 	 [0 1 0 1 0 1 0 1 0 1 0 1] 	 [0 1 2 0 1 2 0 1 2 0 1 2] 	 [0 1 2 3 0 1 2 3 0 1 2 3] 	 [0 1 2 3 4 5 0 1 2 3 4 5]
2: 	 [0 1 0 1 0 1 0 1 0 1 0 1] 	 [0 1 0 0 1 0 0 1 0 0 1 0] 	 [0 1 0 1 0 1 0 1 0 1 0 1] 	 [0 1 0 1 0 1 0 1 0 1 0 1]
3: 	 [0 1 0 1 0 1 0 1 0 1 0 1] 	 [0 1 2 0 1 2 0 1 2 0 1 2] 	 [0 1 2 0 0 1 2 0 0 1 2 0] 	 [0 1 2 0 1 2 0 1 2 0 1 2]
5: 	 [0 1 0 1 0 1 0 1 0 1 0 1] 	 [0 1 2 0 1 2 0 1 2 0 1 2] 	 [0 1 2 3 0 1 2 3 0 1 2 3] 	 [0 1 2 3 4 0 0 1 2 3 4 0]
7: 	 [0 1 0 1 0 1 0 1 0 1 0 1] 	 [0 1 2 0 1 2 0 1 2 0 1 2] 	 [0 1 2 3 0 1 2 3 0 1 2 3] 	 [0 1 2 3 4 5 0 1 2 3 4 5]
11: 	 [0 1 0 1 0 1 0 1 0 1 0 1] 	 [0 1 2 0 1 2 0 1 2 0 1 2] 	 [0 1 2 3 0 1 2 3 0 1 2 3] 	 [0 1 2 3 4 5 0 1 2 3 4 5]
13: 	 [0 1 0 1 0 1 0 1 0 1 0 1] 	 [0 1 2 0 1 2 0 1 2 0 1 2] 	 [0 1 2 3 0 1 2 3 0 1 2 3] 	 [0 1 2 3 4 5 0 1 2 3 4 5]
17: 	 [0 1 0 1 0 1 0 1 0 1 0 1] 	 [0 1 2 0 1 2 0 1 2 0 1 2] 	 [0 1 2 3 0 1 2 3 0 1 2 3] 	 [0 1 2 3 4 5 0 1 2 3 4 5]
19: 	 [0 1 0 1 0 1 0 1 0 1 0 1] 	 [0 1 2 0 1 2 0 1 2 0 1 2] 	 [0 1 2 3 0 1 2 

In [46]:
a = np.arange(1, 25)
for i in range(1, 20):
    print(f"{i}: \t {(a * i) % 2} \t {(a * i) % 3} \t {(a * i) % 7}")

1: 	 [1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0] 	 [1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0] 	 [1 2 3 4 5 6 0 1 2 3 4 5 6 0 1 2 3 4 5 6 0 1 2 3]
2: 	 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 	 [2 1 0 2 1 0 2 1 0 2 1 0 2 1 0 2 1 0 2 1 0 2 1 0] 	 [2 4 6 1 3 5 0 2 4 6 1 3 5 0 2 4 6 1 3 5 0 2 4 6]
3: 	 [1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0] 	 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 	 [3 6 2 5 1 4 0 3 6 2 5 1 4 0 3 6 2 5 1 4 0 3 6 2]
4: 	 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 	 [1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0] 	 [4 1 5 2 6 3 0 4 1 5 2 6 3 0 4 1 5 2 6 3 0 4 1 5]
5: 	 [1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0] 	 [2 1 0 2 1 0 2 1 0 2 1 0 2 1 0 2 1 0 2 1 0 2 1 0] 	 [5 3 1 6 4 2 0 5 3 1 6 4 2 0 5 3 1 6 4 2 0 5 3 1]
6: 	 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 	 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 	 [6 5 4 3 2 1 0 6 5 4 3 2 1 0 6 5 4 3 2 1 0 6 5 4]
7: 	 [1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 